# Canvas Peer Review Tools

Welcome! If you're here, you probably want to use Canvas's built-in Peer Feedback tools but on an assignment that doesn't necessarily look like an essay. Here we've put some helper methods


## Getting Started

To get started you'll need:

* a Canvas API Key ([instructions to generate that can be found here](https://kb.iu.edu/d/aaja))
* your Canvas URL (e.g. https://canvas.northwestern.edu)
* your Course ID (the easiest way to do this is to navigate to your course's Canvas page and copy the 6-digit number at the end of the URL)
* a working Python installation (which you probably have if you're in this notebook)
* to install the `canvasapi` from https://github.com/ucfopen/canvasapi
* to install the `jinja2` templating package

## Why is any of this Necessary?

Canvas's built-in Peer Feedback tools work just fine for essays (think something that fits in a PDF, DOCX, etc.) but not so much for files that require downloading (like a `.py` file). Additionally, though you can assign a rubric to an assignment, it's not necessarilly a particular readable format for the peer getting the feedback. To fix that, we've got some really basic tools here that communicate with Canvas directly via their API.

## What does this assume?

It assumes that you've followed Canvas's instructions to assign a required Peer Review of a particular assignment in the course. [Instructions on how to do that are here](https://community.canvaslms.com/t5/Instructor-Guide/How-do-I-use-peer-review-assignments-in-a-course/ta-p/697).

Unfortunately, there's one manual step here (it can be solved programmatically...but it depends on the exact details of your class).

By default, Canvas will NOT assign a peer review to anyone who didn't complete the assignment themselves. You can override this using Canvas's web interface (i.e. assign them a peer review) **but they won't actually be able to complete it.**

To solve this, you can simply _submit on the student's behalf_. For instance, I just created a blank `.py` file, went through all of the students who did not submit this homework, and then used [Canvas's new "Submit for Student" functionality](https://community.canvaslms.com/t5/Instructor-Guide/How-do-I-submit-an-assignment-on-behalf-of-a-student-as-an/ta-p/560948).

In fact, if you do this BEFORE assigning the Peer Reviews, it will all happen like it's supposed to!

If you want to solve this programmatically you can. I'll include my code for doing it near the bottom of the notebook.

In [ ]:
# Need to install the jinja2 templating system? Run this cell!
%pip install jinja2

In [ ]:
# Need to install the Canvas API Python Wrapper? Run this cell!
%pip install canvasapi

In [1]:
# This is where you can specify your Canvas info
canvas_token = "YOUR_API_TOKEN_GOES_HERE"
course_id = 123456
canvas_url = "https://canvas.cool-place.edu"

In [ ]:
# These will be the main instances of your Canvas connection and course
# More info in the canvasapi docs: https://canvasapi.readthedocs.io/en/stable/
canvas = Canvas(canvas_url, canvas_token)
current_course = canvas.get_course(course_id)

In [ ]:
from canvasapi.submission import Submission
from canvasapi.upload import FileOrPathLike, Uploader

# Temporary fix to make sure comments/feedback go to latest submission
#   and so you can post file_commments with custom text
#   Have submitted PR to ucfopen/canvasapi to get this fixed
def upload_comment(self, file: FileOrPathLike, **kwargs):
    response = Uploader(
        self._requester,
        "courses/{}/assignments/{}/submissions/{}/comments/files".format(
            self.course_id, self.assignment_id, self.user_id
        ),
        file,
        **kwargs,
    ).start()

    if response[0]:
        if "comment" in kwargs:
            kwargs["comment"] |= {"file_ids": [response[1]["id"]]}
        else:
            kwargs["comment"] = {"file_ids": [response[1]["id"]]}

        self.edit(**kwargs)

    return response


# monkey patching built-in Submission.upload_comment with the new version
Submission.upload_comment = upload_comment

In my class, students were doing a peer review of one assignment and it was counting for a grade on another assignment. Specifically:

* HW 3 - The assignment that was going to be peer reviewed
* HW 4 - The assignment which contained review instructions and would hold the grade for whether or not the student completed the review

So first, we need to get the `ID`s for each of these assignments (a unique numeric code that identifies those assignments in Canvas).

In [ ]:
# Run this cell to see all assignments in your Canvas course
# Note, this does NOT include Quizzes as those are not technically Assignments
all_assignments = current_course.get_assignments()

for an_assignment in all_assignments:
    print(an_assignment)

In [ ]:
# If you're following a similar structure to my setup, insert your assignment ids here
hw3 = current_course.get_assignment(ASSIGNMENT_ID_FOR_HW3)
hw4 = current_course.get_assignment(ASSIGNMENT_ID_FOR_HW4)

In [ ]:
hw4_subs = {}
for sub in hw4.get_submissions():
    if sub.grade:
        print('already graded')
    else:
        hw4_subs[str(sub.id)] = sub

In [ ]:
hw4_subs[str(43099227)]

In [ ]:
subs = hw3.get_submissions(include="peer_assessments", style="full")

In [ ]:
# Unfortunately, the Canvas feedback interface doesn't allow a student to download the other's submission. 
#   To fix this, we just post that student's submission (with names removed) as a file comment on their reviewer's submission.

# Keep track of who's been taken care of already just in case we mess up partway
done = []

In [ ]:
# Loop through each peer review
for item in hw3.get_peer_reviews(include="submission_comments"):
    print(item.assessor_id, item.user_id, item.workflow_state) # progress statement
    
    # Double check we haven't taken care of this one already
    if item.assessor_id in done:
        print("Skipping!")
        continue
        
    # Get the reviewee and the reviewer
    reviewee = current_course.get_user(item.user_id)
    reviewer = current_course.get_user(item.assessor_id)
    
    # Get the submission for HW 3 for the reviewee
    sub_to_be_reviewed = hw3.get_submission(item.user_id)
    
    # Get the submission for HW 4 for the reviewer
    sub_for_reviewer = hw4.get_submission(item.assessor_id)
    # Check to see if they have any attachments as part of their submission
    if hasattr(sub_to_be_reviewed, "attachments"):
        if len(sub_to_be_reviewed.attachments) > 1:
            print("MULTIPLE ATTACHMENTS WARNING")
        for i in range(len(sub_to_be_reviewed.attachments)):
            filename = f"hw3_peer_review_for_{reviewer.name.replace(' ', '_')}.py"
            
            # Actually download the submission
            try:
                print(f"    Downloading...{filename}")
                sub_to_be_reviewed.attachments[i].download(filename)
                
                # Create a comment to post to the reviewer's HW 4
                comment_dict = {
                    "attempt": sub_for_reviewer.attempt,
                    "text_comment": 
                        "Here's a link to the file for your peer's submission! You can download this file and put it in the same folder as your own HW3 to run it and see your peer's creature!"
                }
                
                # Upload the reviewee's submission to the reviewer's HW4.
                sub_for_reviewer.upload_comment(filename, comment=comment_dict)
                print(f"Uploaded {filename} for {reviewer.name} who's peer reviewing {reviewee.name}")
                done.append(reviewer.id)
                
            except Exception as e:
                print("    Couldn't download assignment: {}".format(e))

## What's happened so far?

So at this point, we've "posted" everyone's HW 3 submission as a file comment on their reviewers HW 4 submission! Why is this important? Because now each reviewer has download access to each reviewee's file!

## What next?

Well, now you wait for everyone to complete their Peer Review! Some caveats:

* Canvas **does not** timestamp Peer Reviews in any way that we have access to
* If you want to set a deadline, then you'll need to grade these reviews right at the deadline
* If you want to allow late peer reviews, then you'll have to track those "manually"

## What now?

Alright so everyone's completed their review. How do you...
* Grade it (maybe with some requirements on the comments)
* Make it available to the Peer (beyond the default Canvas rubric which will)
* Make a copy of it available to the reviewer

Well, let's do it!

In [ ]:
# Inside a Peer Review stored in Canvas, we have a mapping of "reviewer" to "reviewee".
#   While that's inside the peer review, it's NOT inside the actual "rubric assessment" object
#   that gets affiliated with each student's submission. So we save these relations in a "map"
#   which is just a dictionary mapping a reviewer's id to their reviewee's id.
reviewer_to_reviewee = {}
for item in hw3.get_peer_reviews():
    reviewer_to_reviewee[str(item.assessor_id)] = item.user_id

In [ ]:
# If you're using a rubric, we need to find its id. This cell will just print out
#   all of the rubrics with their names and ids so you can identify the one you want.
rubrics = current_course.get_rubrics()
for rubric in rubrics:
    print(rubric)

In [ ]:
## Grab the specific rubric you found
canvas_rubric = current_course.get_rubric(RUBRIC_ID_GOES_HERE, include="peer_assessments", style="full")

# This fetches the raw Canvas rubric which is documented here:
#   https://canvas.instructure.com/doc/api/rubrics.html

# It then loops through that rubric and creates a more Pythonic version of it to be used later
py_rubric = {}
for rubric_item in canvas_rubric.data:
    py_rubric[rubric_item['id']] = {'description': rubric_item['description'], 'long_description': rubric_item['long_description']}
    
## Have to map reviewers and reviewees cause the way Python stores the rubrics
reviewer_to_reviewee = {}
for item in hw3.get_peer_reviews():
    reviewer_to_reviewee[str(item.assessor_id)] = item.user_id

# Load in the peer review HTML template (a prettier version of the Canvas rubric)
from jinja2 import Environment, FileSystemLoader
environment = Environment(loader=FileSystemLoader("./"))
template = environment.get_template("templates/peer_review_template.html")

# Go through every "rubric assessment" (peer review)
for pr in canvas_rubric.assessments:    
    rubric = {}
    
    comment_penalty = 0
    
    # look up the reviewer and reviewee (using that mapping we created earlier)
    reviewer = current_course.get_user(pr['assessor_id'])
    reviewee = current_course.get_user(reviewer_to_reviewee[str(pr['assessor_id'])])

    # load their submissions, HW3 for the reviewee; HW4 for the reviewer; and HW4 for the reviewee
    sub_that_was_reviewed = hw3.get_submission(reviewee.id)
    sub_for_reviewer = hw4.get_submission(reviewer.id)
    reviewee_hw4_sub = hw4.get_submission(reviewee.id)
  
    # Check to make sure this reviewer hasn't already received a grade
    if sub_for_reviewer.workflow_state == "graded":
        print("Already graded!")
        continue
    
    # Process the reviewer's  review
    for item in pr['data']:
        
        # Data fix for "blank" comments
        if item['comments'] is None:
            item['comments'] = ""
            
        
        ## Here's where you would place "expectations" for reviewer comments.
        ##  For instance, I asked that reviewers add at least two sentences to each
        ##  rubric item justifying their choice. I check for this by expecting each
        ##  rubric item to have at least 10 words in its comment property.
        ##  You can implement anything here – even expectations on word usage, piping it
        ##  through an LLM or other automated language model (simple or advanced)
            
        # Do word count check
        word_count = len(re.findall(r'\w+', item['comments']))
        
        # If the word count is below the expectation, they didn't meet the expectations
        if word_count < 10:
            comment_penalty += 1
            
        rubric[py_rubric.get(item['criterion_id'])['description']] = {'description': item['description'], 
                                                                      'long_description': py_rubric.get(item['criterion_id'])['long_description'], 
                                                                      'comments':item['comments'], 
                                                                      'class': 'warning' if item['description'] == "Room for Improvement" else 'success'}    
    # Everyone starts with the max grade
    grade = hw4.points_possible
    # If they received comment penalties, their grade goes down.
    # In my case, I spotted everyone "1" comment penalty
    if comment_penalty > 1:
        grade = grade - (comment_penalty - 1) * 10
        
    # Output to the screen who gets what (this is all local so deanonymized).
    print(f"{reviewer.short_name} will receive a {grade} for their review of {reviewee.short_name}")
    
    # Generate anonymous file names for the reviewer (who gets a copy) and the reviewee
    file_for_reviewer = f"HW3_Peer_Review_by_{reviewer.short_name.replace(' ', '_')}.html"
    file_for_reviewee = f"HW3_Peer_Review_for_{reviewee.short_name.replace(' ', '_')}.html"
    
    # Render the two peer reviews in our nice template
    with open(file_for_reviewer, mode="w", encoding="utf-8") as report:
        content = template.render(
                title= file_for_reviewer[:-5].replace("_", " "),
                student_name=reviewer.short_name,
                netid=reviewer.sis_user_id,
                rubric=rubric
                )
        report.write(content)
    
    with open(file_for_reviewee, mode="w", encoding="utf-8") as report:
        content = template.render(
                title= file_for_reviewee[:-5].replace("_", " "),
                student_name=reviewee.short_name,
                netid=reviewee.sis_user_id,
                rubric=rubric
                )
        report.write(content)
    

    # GRADE Peer Review for REVIEWER AND POST REVIEWER FILE    
    sub_for_reviewer.upload_comment(
                file_for_reviewer,
                submission={"posted_grade": grade},
                comment={
                    "attempt": sub_for_reviewer.attempt,
                    "text_comment": "Thanks for submitting a HW 3 Peer Review! Here's the contents of your review:"
                },
            )
    
    
    # POST Reviewee FILE to REVIEWEE SUB
    reviewee_hw4_sub.upload_comment(
                file_for_reviewee,
                comment={
                    "attempt": reviewee_hw4_sub.attempt,
                    "text_comment": "Here's a nicer version of your Peer's Review of your HW 3 submission!",
                },
            )

## That's it!

That's basically it. Not so bad! Frustrating you have to do all of that yourself, but oh well. 


## Issues

So...as in all things...there are some issues here.

* Canvas Peer Reviews, once submitted, are uneditable.

If you want students to be able to edit their reviews, resubmit, etc., you'll have to do that through some other mechanism. While the Canvas Web interface allows you to "unassign" a peer review...it doesn't actually delete the person's "rubric assessment" meaning if you assign them the same peer to review...you've made no progress towards resetting the peer review! So this means you need to manually go in and delete the rubric association, something the `canvasapi` library does not support so you need to make a custom request. More details below.

* Students don't follow instructions. What do you do if they put the comments not in the rubric but the assignment itself?

Good question to which I do not have an answer. If you take a look at my instructions, I even included a screenshot of how to add comments in the rubric...and yet.

Additionally, it's worth noting that comments added to the SUBMISSION rather than the RUBRIC are NOT anonymous. So there's so identity leakage here. In my class, I decided it wasn't worth the fight to take points off for this and just reviewed these manually (i.e. anyone who didn't get a 100, I looked at in the Canvas web interface to see what they did wrong).

It would be possible to use the Canvas API to parse an overall comment, delete it, and move it into a rubric, but it would be non-trivial. I don't think it's worth the time necessary to figure out when you can just tell the student to do it again (use the delete steps below) or assign them a penalty for not following the instructions.

In [ ]:
# You'll need the reviewer's Canvas ID. You can find that via the API, or you can go to that
#  person's Canvas profile in your course and grab it from the URL.
REIVEWER_ID_TO_DELETE = 1234567

## THIS WILL DELETE A RUBRIC ASSESSMENT
pr_rubric = current_course.get_rubric(RUBRIC_ID, include=["peer_assessments"])
for i in pr_rubric.assessments:
    if i['assessor_id'] == REIVEWER_ID_TO_DELETE:
        print("found reviewer's rubric sumission")
        break
        
# Interestingly...I've gotten some odd behavior doing this. Sometimes it seems to destroy the peer review entirely
#.  meaning you have to go and assign the review again.
print(f"courses/{course_id}/rubric_associations/{i['rubric_association_id']}/rubric_assessments/{i['id']}")
pr_rubric._requester.request("DELETE", f"courses/{course_id}/rubric_associations/{i['rubric_association_id']}/rubric_assessments/{i['id']}")

# Now go see if you need to go re-assign the peer review. If you do, here's their details:
reviewer = current_course.get_user(REIVEWER_ID_TO_DELETE)
peer = current_course.get_user(reviewer_to_reviewee.get(str(REIVEWER_ID_TO_DELETE)))
print(reviewer.short_name, "needs to be re-assigned to review", peer.short_name)